In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from os import listdir

In [2]:
def read_all():
    df = pd.concat([pd.read_json(f'data/{f}') for f in listdir('data') if f.endswith('.json')]).reset_index(drop=True)
    return df

def process_data(df):
    items = df['CVE_Items']
    
    def _clean(item):
        
        values = {
            'cve_id': item['cve']['CVE_data_meta']['ID']
        }
        
        # Add metrics
        for version in ['2', '3']:
            if f'baseMetricV{version}' in item['impact']:
                v = item['impact'][f'baseMetricV{version}'].copy()
                cvss = v.pop(f'cvssV{version}')
                values.update({f'V{version}_{key}': value for key, value in v.items()})
                values.update({f'V{version}_{key}': value for key, value in cvss.items()})
        
        return pd.Series(values)
    
    return items.apply(_clean)

In [3]:
src_data = read_all()

In [4]:
df = process_data(src_data)

In [5]:
df.head()

,cve_id,V2_severity,V2_exploitabilityScore,V2_impactScore,V2_obtainAllPrivilege,V2_obtainUserPrivilege,V2_obtainOtherPrivilege,V2_userInteractionRequired,V2_version,V2_vectorString,...,V3_attackVector,V3_attackComplexity,V3_privilegesRequired,V3_userInteraction,V3_scope,V3_confidentialityImpact,V3_integrityImpact,V3_availabilityImpact,V3_baseScore,V3_baseSeverity
0,CVE-2011-0001,MEDIUM,10.0,2.9,False,False,False,False,2.0,AV:N/AC:L/Au:N/C:N/I:N/A:P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CVE-2011-0002,MEDIUM,10.0,4.9,False,False,False,False,2.0,AV:N/AC:L/Au:N/C:P/I:P/A:N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CVE-2011-0003,MEDIUM,8.6,4.9,False,False,False,True,2.0,AV:N/AC:M/Au:N/C:P/I:P/A:N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CVE-2011-0004,MEDIUM,8.6,2.9,False,False,False,True,2.0,AV:N/AC:M/Au:N/C:N/I:P/A:N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CVE-2011-0005,MEDIUM,8.6,2.9,False,False,False,True,2.0,AV:N/AC:M/Au:N/C:N/I:P/A:N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.to_parquet('data/df_v1.pqt')

In [7]:
# Now values that are only have valid CVEs
only_valid = df[~df['V2_impactScore'].isna()].reset_index()
only_valid.to_parquet('data/df_v2.pqt')

In [8]:
df.shape

(161262, 32)

In [9]:
df.describe()

,V2_exploitabilityScore,V2_impactScore,V2_baseScore,V3_exploitabilityScore,V3_impactScore,V3_baseScore
count,151637.000000,151637.000000,151637.000000,78264.000000,78264.000000,78264.000000
mean,8.134079,5.541438,5.977483,2.712022,4.388922,7.240122
std,2.162143,2.622113,2.001626,0.944379,1.509976,1.652467
min,1.200000,0.000000,0.000000,0.100000,1.400000,1.800000
25%,8.000000,2.900000,4.300000,1.800000,3.600000,6.100000
50%,8.600000,6.400000,5.500000,2.800000,3.600000,7.500000
75%,10.000000,6.400000,7.500000,3.900000,5.900000,8.800000
max,10.000000,10.000000,10.000000,3.900000,6.000000,10.000000


## Viewing the number of CVEs that have V2 and V3 metrics

In [10]:
(~df['V2_impactScore'].isna()).sum(), (~df['V3_impactScore'].isna()).sum()

(151637, 78264)

### Explore the rows that have missing data

In [11]:
missing = df[df['V2_impactScore'].isna()].index

In [12]:
src_data.loc[missing]

,CVE_data_type,CVE_data_format,CVE_data_version,CVE_data_numberOfCVEs,CVE_data_timestamp,CVE_Items
42,CVE,MITRE,4,4813,2021-04-13T08:23Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
57,CVE,MITRE,4,4813,2021-04-13T08:23Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
91,CVE,MITRE,4,4813,2021-04-13T08:23Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
95,CVE,MITRE,4,4813,2021-04-13T08:23Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
96,CVE,MITRE,4,4813,2021-04-13T08:23Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
...,...,...,...,...,...,...
161209,CVE,MITRE,4,5002,2021-04-13T08:27Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
161210,CVE,MITRE,4,5002,2021-04-13T08:27Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
161211,CVE,MITRE,4,5002,2021-04-13T08:27Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."
161212,CVE,MITRE,4,5002,2021-04-13T08:27Z,"{'cve': {'data_type': 'CVE', 'data_format': 'M..."


In [13]:
src_data.loc[missing]['CVE_Items'].apply(lambda x: x['cve']['description']['description_data'])

42        [{'lang': 'en', 'value': '** REJECT **  DO NOT...
57        [{'lang': 'en', 'value': '** REJECT **  DO NOT...
91        [{'lang': 'en', 'value': '** REJECT **  DO NOT...
95        [{'lang': 'en', 'value': '** REJECT **  DO NOT...
96        [{'lang': 'en', 'value': '** REJECT **  DO NOT...
                                ...                        
161209    [{'lang': 'en', 'value': '** REJECT ** DO NOT ...
161210    [{'lang': 'en', 'value': '** REJECT ** DO NOT ...
161211    [{'lang': 'en', 'value': '** REJECT **  DO NOT...
161212    [{'lang': 'en', 'value': '** REJECT **  DO NOT...
161249    [{'lang': 'en', 'value': '** REJECT ** DO NOT ...
Name: CVE_Items, Length: 9625, dtype: object

### Get the first 12 characters of the value of those that were not correct

In [14]:
length_of_one = (src_data.loc[missing]['CVE_Items'].apply(lambda x: len(x['cve']['description']['description_data'])) == 1)
src_data.loc[missing][length_of_one]['CVE_Items'].apply(lambda x: x['cve']['description']['description_data'][0]['value'][:12]).value_counts(normalize=True)

** REJECT **    0.958785
Remote Proce    0.002817
Multiple vul    0.001565
A vulnerabil    0.001356
An improper     0.001148
                  ...   
A clear text    0.000104
ZEROF Web Se    0.000104
Improper han    0.000104
Some ZTE pro    0.000104
Windows Reso    0.000104
Name: CVE_Items, Length: 226, dtype: float64

### Since a large number of those were rejected, we skip them from the rest of the analysis

In [15]:
impact_vars = ['V2_confidentialityImpact', 'V2_integrityImpact', 'V2_availabilityImpact', 'V3_confidentialityImpact', 'V3_integrityImpact', 'V3_availabilityImpact']
df[impact_vars]

,V2_confidentialityImpact,V2_integrityImpact,V2_availabilityImpact,V3_confidentialityImpact,V3_integrityImpact,V3_availabilityImpact
0,NONE,NONE,PARTIAL,NaN,NaN,NaN
1,PARTIAL,PARTIAL,NONE,NaN,NaN,NaN
2,PARTIAL,PARTIAL,NONE,NaN,NaN,NaN
3,NONE,PARTIAL,NONE,NaN,NaN,NaN
4,NONE,PARTIAL,NONE,NaN,NaN,NaN
...,...,...,...,...,...,...
161257,NONE,NONE,PARTIAL,NONE,NONE,HIGH
161258,COMPLETE,COMPLETE,COMPLETE,HIGH,HIGH,HIGH
161259,COMPLETE,COMPLETE,COMPLETE,HIGH,HIGH,HIGH
161260,NONE,PARTIAL,NONE,LOW,LOW,NONE


In [16]:
impact = df[['cve_id'] + impact_vars]

## Check if impact values can be mapped from V2 to V3
First we get the rows that are valid for both v2 and v3

In [17]:
valid = impact.dropna()
valid.shape

(78264, 7)

### confidentiality
If they can be mapped, then no value of V2 should appear with different values of V3 and viceversa

In [21]:
valid.groupby(['V2_confidentialityImpact', 'V3_confidentialityImpact']).size().rename('number_of_cves').to_frame()

number_of_cves
V2_confidentialityImpact V3_confidentialityImpact                
COMPLETE                 HIGH                               13055
                         LOW                                    7
                         NONE                                   4
NONE                     HIGH                                 135
                         LOW                                 8994
                         NONE                               16953
PARTIAL                  HIGH                               32956
                         LOW                                 6102
                         NONE                                  58

We see that the mapping is not direct, although COMPLETE V2 confidentiality impacts tend to have a HIGH V3 value 

### integrity
If they can be mapped, then no value of V2 should appear with different values of V3 and viceversa

In [22]:
valid.groupby(['V2_integrityImpact', 'V3_integrityImpact']).size().rename('number_of_cves').to_frame()

number_of_cves
V2_integrityImpact V3_integrityImpact                
COMPLETE           HIGH                         12731
                   LOW                              1
                   NONE                             4
NONE               HIGH                          1177
                   LOW                             51
                   NONE                         24368
PARTIAL            HIGH                         26197
                   LOW                          13707
                   NONE                            28

The same occurs for the integrity values

### availability
If they can be mapped, then no value of V2 should appear with different values of V3 and viceversa

In [23]:
valid.groupby(['V2_availabilityImpact', 'V3_availabilityImpact']).size().rename('number_of_cves').to_frame()

number_of_cves
V2_availabilityImpact V3_availabilityImpact                
COMPLETE              HIGH                            15352
                      LOW                                 4
                      NONE                               54
NONE                  HIGH                             1237
                      LOW                                63
                      NONE                            29815
PARTIAL               HIGH                            29206
                      LOW                              1987
                      NONE                              546

The same occurs for the V2 COMPLETE values, although for availability, in V3 most of them, if defined, go to HIGH